In [9]:
%matplotlib inline
import os
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import SimpleRNN

from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import math
from sklearn.metrics import mean_squared_error
import FinanceDataReader as fdr
from sklearn.metrics import accuracy_score

In [2]:
#특성 추가 ------------------------------------------
    #이동평균선
def get_MA(df):
    MA_26=df["Close"].rolling(26).mean()
    MA_52=df["Close"].rolling(52).mean()
    df=df.assign(MA_26=MA_26,MA_52=MA_52).dropna()
    
    return df

    
    #스토캐스틱
def get_stochastic(df, n=15, m=5, t=3):
    # n일중 최고가
    ndays_high = df.High.rolling(window=n, min_periods=1).max()
    # n일중 최저가
    ndays_low = df.Low.rolling(window=n, min_periods=1).min()
 
    # Fast%K 계산
    kdj_k = ((df.Close - ndays_low) / (ndays_high - ndays_low))*100
    # Fast%D (=Slow%K) 계산
    kdj_d = kdj_k.ewm(span=m).mean()
    # Slow%D 계산
    kdj_j = kdj_d.ewm(span=t).mean()
 
    # dataframe에 컬럼 추가
    df = df.assign(kdj_k=kdj_k, kdj_d=kdj_d, kdj_j=kdj_j).dropna()
    
    return df
   
    #시간
def get_time(df):
    time=np.linspace(0,10,len(df),endpoint=False).reshape(-1,1)
    df=df.assign(time=time)
    
    return df

In [3]:
a=fdr.DataReader("KS11","2010-01-01")

In [10]:
KS11 = fdr.DataReader("KS11","2010-01-01","2019-01-01")
KQ11= fdr.DataReader("KQ11","2010-01-01","2019-01-01")
US500 = fdr.DataReader("US500","2010-01-01","2019-01-01")
HSI = fdr.DataReader("HSI","2010-01-01","2019-01-01")
IXIC = fdr.DataReader("IXIC","2010-01-01","2019-01-01")

In [77]:
a["Close"]

(640,)

In [79]:
import time
start = time.time()
Xtrain_size=0
예측dic={}
수익률dic={}
END=0
초기돈=10000
d=1
dd=20
BATCH = 20
투자결과=[]
총df=pd.DataFrame(columns=["영업일","인덱스","상승예측확률","실제수익률"])
세트df=pd.DataFrame(columns=["영업일","인덱스","상승예측확률","실제수익률"])


  
for 인덱스,인덱스명 in zip([KS11,KQ11,IXIC,HSI,US500],["KS11","KQ11","IXIC","HSI","US500"]):


    df = 인덱스
    df=get_MA(df)
    df=get_stochastic(df)
    df=df[0:-(len(df)%BATCH)]
    #-----------------------------------------------
    
    # 값과 이평선 비교하기 ----------------
    MA_26=df["Close"].rolling(26).mean()
    y_before=MA_26.dropna()
    df_before=df["Close"][25:-20]
    #예측머신--------------------------------------------------------
    
    
    df=df[len(df)%20:]
    X=df[0:-20]                             # 값일떈 0 이평선일땐 25 슬라이스 필요
    dfA=X
    
    #y=np.where(np.array(df_before)>np.array(y_before[20:]),1,0) #값과 이평선
    y=np.where(df[20:]["Change"]>0,1,0) #값과 값
    
    df = X.values
    df.astype('float32')
    scaler = MinMaxScaler(feature_range=(0, 1))
    df = scaler.fit_transform(df)
    X=df

    setlen=int(len(X)/20*0.7)*20

    X_train=X[:setlen]
    X_test=X[setlen:]
    y_train=y[:setlen]
    y_test=y[setlen:]

    X_train = np.reshape(X_train, (X_train.shape[0],1, X_train.shape[1]))
    X_test = np.reshape(X_test, (X_test.shape[0],1, X_test.shape[1]))

    model = Sequential()
    model.add(LSTM(36, input_shape=(1, 11)))
    for i in range(3):
        model.add(Dense(36,activation='sigmoid'))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
    model.fit(X_train, y_train, epochs=100, batch_size=16,validation_split=0.1,verbose=0)
    pred=np.where(model.predict(X_test)>0.5,1,0)
    
    print(인덱스명, "정확도 : ", accuracy_score(y_test,np.where(model.predict(X_test)>0.5,1,0)))
    예측dic[인덱스명]=pred
    수익률dic[인덱스명]=dfA["Close"][setlen:]

    #예측머신 끝 --------predict -> 12근방 나옴 ---------------------


        
        
버튼=1
date=0

while 버튼==1:#예측머신돌리기
    day= str(d+(20*date)) +"~" + str(dd+(20*date)) + "일 차"
    date=date+1
    print(day,"---------------------------------")

    세트df=pd.DataFrame(columns=["영업일","인덱스","상승예측확률","실제수익률"])
    
    for x,y in zip(예측dic.keys(),예측dic.values()):
          
           #종료 조건
        if(len(수익률dic[x])==BATCH+20):
            버튼=0
            print("종료")
            break
        
    
        pred=y[BATCH-20:BATCH]
        예측 = (sum(pred))/20
        실제수익률 = ((수익률dic[x][BATCH+20])/(수익률dic[x][BATCH]))-1
        턴df=pd.DataFrame(data={"영업일":[day],"인덱스":[x],"상승예측확률":[예측],"실제수익률":[실제수익률] }, columns=["영업일","인덱스","상승예측확률","실제수익률"])
        세트df=pd.concat([세트df,턴df])    
    
    if 버튼==0:
        break
        
        
    earn_list=[]             
    for x,y in zip(세트df["상승예측확률"],세트df["실제수익률"]):
        
        if x>=0.6:
            earn_list.append(y)

    earn_list2=[]
    if len(earn_list)!=0:
        초기돈=초기돈/(len(earn_list))
        for y in earn_list:
            earn_list2.append(초기돈+(초기돈*y))
        초기돈=sum(earn_list2)    
    else:
        earn_list2.append(초기돈)


    총df=pd.concat([총df,세트df])
    BATCH=BATCH+20

    print("분산 투자 : ",earn_list2)
    print("투자 결과 : ",초기돈)
    print("\n★")
    print(세트df)
    print("★\n")
    
   
            #MONEY = 투잣돈 * A의 20일후 change + 투잣돈 * B의 20일후 change #"20일차MONEY"출력


KS11 정확도 :  0.4863636363636364
KQ11 정확도 :  0.5318181818181819
IXIC 정확도 :  0.5651515151515152
HSI 정확도 :  0.4818181818181818
US500 정확도 :  0.48333333333333334
1~20일 차 ---------------------------------
분산 투자 :  [2494.5019404915906, 2582.794804998995, 2331.9937588165394, 2540.8657337917007]
투자 결과 :  9950.156238098825

★
       영업일    인덱스 상승예측확률     실제수익률
0  1~20일 차   KS11  [0.0] -0.001781
0  1~20일 차   KQ11  [1.0] -0.002199
0  1~20일 차   IXIC  [1.0]  0.033118
0  1~20일 차    HSI  [1.0] -0.067202
0  1~20일 차  US500  [1.0]  0.016346
★

21~40일 차 ---------------------------------
분산 투자 :  [2513.5550693957025, 2316.575613841401, 2550.5383589393323, 2370.785680324315]
투자 결과 :  9751.45472250075

★
        영업일    인덱스 상승예측확률     실제수익률
0  21~40일 차   KS11  [0.1]  0.022268
0  21~40일 차   KQ11  [1.0]  0.010459
0  21~40일 차   IXIC  [1.0] -0.068728
0  21~40일 차    HSI  [1.0]  0.025326
0  21~40일 차  US500  [1.0] -0.046935
★

41~60일 차 ---------------------------------
분산 투자 :  [2391.708212968112, 2711.452386183896, 

분산 투자 :  [6443.841570428534, 6890.280394524303]
투자 결과 :  13334.121964952838

★
          영업일    인덱스 상승예측확률     실제수익률
0  601~620일 차   KS11  [0.0] -0.069876
0  601~620일 차   KQ11  [1.0] -0.119875
0  601~620일 차   IXIC  [1.0] -0.058898
0  601~620일 차    HSI  [0.0] -0.056241
0  601~620일 차  US500  [0.0] -0.047774
★

621~640일 차 ---------------------------------
종료


In [4]:
# 면역전략의 수익
list=[]
for 인덱스,인덱스명 in zip([KS11,KQ11,IXIC,HSI,US500],["KS11","KQ11","IXIC","HSI","US500"]):
    돈 = 10000
    df=인덱스
    
    df=df[25:]
    df=df[len(df)%20:]
    setlen=int(len(X)/20*0.7)*20
    df=df[setlen:]
    
    print(인덱스명," : " , 돈 + ( 돈 * (df["Close"][-1] - df["Close"][0])/ df["Close"][0] ))
    print("수익률 : ",((df["Close"][-1] - df["Close"][0])/ df["Close"][0] ) *100 , "%\n")
    list.append((df["Close"][-1] - df["Close"][0])/ df["Close"][0] )
    
print("분산 투자시")
list2=[]
for x in list:
    print(2000+(2000*x))
    list2.append(2000+(2000*x))
print("\n총 수익 : ",sum(list2))

NameError: name 'KS11' is not defined

In [45]:
X_train = np.reshape(X_train, (int(X_train.shape[0]/BATCH),BATCH, X_train.shape[1]))[:,:19,:]

ValueError: cannot reshape array of size 20880 into shape (4,20,20)

In [19]:
y_train.shape

(84, 1)

SyntaxError: can't assign to function call (<ipython-input-39-0b6f1e348eaf>, line 2)

In [11]:
# 역사적 변동성 추가 -----------------
인덱스=KS11
import math
list=[] #로그수익률
for xx in range(인덱스.shape[0]-1):
    math.log(인덱스["Close"][xx+1] / 인덱스["Close"][xx])
    average = sum(list)/len(list) # 로그수익률의 평균

list2=[] # 로그수익률의 편차의 제곱

for xx in range(인덱스.shape[0]-1):
    list2.append((average - (math.log(인덱스["Close"][xx+1] / 인덱스["Close"][xx]))) **2)
average2 = sum(list2)/len(list2) #편차의 제곱의 평균

V=math.sqrt(252*average2) # 일별변동성

list3=[] # 그냥 표준편차
for xx in 인덱스["Change"]:
    xx=abs(xx)
    list3.append(math.sqrt(xx)/V) # 표준편차/일별변동성

인덱스 = 인덱스.assign(V=list3)



In [14]:
KS11

,Close,Open,High,Low,Volume,Change
Date,,,,,,
2010-01-04,1696.14,1681.71,1696.14,1681.71,296550000.0,0.0079
2010-01-05,1690.62,1701.62,1702.39,1686.45,408850000.0,-0.0033
2010-01-06,1705.32,1697.88,1706.89,1696.10,426040000.0,0.0087
2010-01-07,1683.45,1702.92,1707.90,1683.45,462400000.0,-0.0128
2010-01-08,1695.26,1694.06,1695.26,1668.84,379950000.0,0.0070
...,...,...,...,...,...,...
2018-12-21,2061.49,2052.70,2061.51,2049.76,311390000.0,0.0007
2018-12-24,2055.01,2050.38,2059.94,2046.18,285280000.0,-0.0031
2018-12-26,2028.01,2028.81,2037.83,2014.28,321500000.0,-0.0131
